In [1]:
#This notebook contains each of the Wichmann and Tsunami catalog
#observation distributions, their reference location and a brief 
#explanation of why we constructed them the way we did

"""
Created By Cody Kesler and Hunter
Property of BYU Mathematics Dept.

** Need to install openpyxl and xlsxwriter **
"""
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

import sys
sys.path.append('../Classes_PreRun')
from Gauge import Gauge
from ShoreAngle import ShoreAngle
gauges = list()

In [2]:
#EXAMPLE GAUGE. COPY AND FILL IN FOR AS MANY GAUGES AS YOU HAVE:
city_name = 'Pulu Ai - Wichmann'
name = 10001 # Pulu Ai - Wichmann
longitude = -4.517863
latitude = 129.7745653
distance = 2.5 # in kilometers (max 5) //TODO Not sure what this is??

# Order for 
kind = ['skewnorm', 'norm', 'chi2']
# Mix and match with different distributions as defined above.
# *****BUT KEEP IN ORDER OF Arrival, Wave Height, and Inundation****

# 1. ARRIVAL 
# For kind = 'skewnorm'
arrival_skew_param = None
arrival_mean = None  # in minutes
arrival_std = None
arrival_params = [arrival_skew_param, arrival_mean, arrival_std]

# 2. Wave Height
# For kind = 'norm'
height_mean = 1. # in meters
height_std =  0.2
height_params = [height_mean, height_std]

# 3. Inundation
# For kind = 'chi2'
inundation_k = None
inundation_lower_bound = None
inundation_params = [inundation_k, inundation_lower_bound]

# USE IF YOU HAVE SHORE PROFILES ANGLES IN AN EXCEL SHEET GENERATED FROM () :
# just the name of the excel file with no extention:
ShoreAngles = ShoreAngle('lonthor_profiles') 
profiles_to_average = [10,11,12,13]
LONTHOR_average_angle = ShoreAngles.getAveragesSlopeAngles(profiles_to_average)
beta = LONTHOR_average_angle

# 
n = .03

# Create gauge for use by Geoclaw
gauges.append(Gauge(name, longitude, latitude, distance, kind, arrival_params, height_params, 
          inundation_params, beta, n, city_name).to_json())


#THIS IS FOR DISPLAYING THE DISTRIBUTIONS IF NEEDED:
# Skewnorm
domain = np.linspace(0,600,1000)
distrib = stats.skewnorm.pdf(domain, _skew_param, _mean,_std)
plt.plot(domain,distrib)
plt.xlabel("Appropriate Label")
plt.show()

# Normal Distribution
dist = stats.norm(_mean, _std)
domain = np.linspace(0,8,1000)
plt.plot(domain, dist.pdf(domain))
plt.xlabel("Appropriate Label")
plt.plot()
plt.show()

# Chi2
domain = np.linspace(_lower_bound, _upper_bound,100)
plt.plot(domain, stats.chi(height_k).pdf(domain - .5))
plt.xlabel("Appropriate Label")
plt.show()

Arrival/Height/Inundation Distribution Justification (FOR ABOVE): Explain why you chose the numbers you did

In [ ]:
#Output gauges
# print(gauges)
np.save('../../InputData/gauges.npy', gauges)


AMR_levels =  [2, 2, 3, 5]


####  WRITE MODEL_BOUNDS.TXT FILE   ####
g = open('../../InputData/model_bounds.txt', 'w')
g.write("{\"xlower\": 125.0, ") 
g.write("\"xupper\": 134.5, ")      
g.write( "\"ylower\": -9.5, " )
g.write( "\"yupper\": -2.5, " )
g.write("\"run_time\": 90.0, " )
g.write( "\"xcoarse_grid\": 95," )
g.write( "\"ycoarse_grid\": 70," )
g.write( "\"gauge_topo\": "
        "[\"banda_map_merged.tt3\", "
        "\"gauge10002_merged3.tt3\", "
        "\"gauge10004_merged3.tt3\", "
        "\"gauge10005_merged3.tt3\", "
        "\"gauge10006_merged3.tt3\", "
        "\"5_merged.tt3\", "
        "\"6_merged.tt3\"], "  )
g.write( "\"regions_bounds\": "
        "[[300, 1e10, 129.86, 129.95, -4.576, -4.49], " 
        "[300, 1e10, 129.76,129.785 , -4.525, -4.515], "
        "[1200, 1e10, 127.98, 128.2, -3.8, -3.66], "
        "[1800, 1e10, 127.05, 127.28, -3.39, -3.27], "
        "[1200, 1e10, 128.54, 128.565, -3.557, -3.53], "
        "[1200, 1e10, 128.652, 128.7, -3.6, -3.573], "
        "[1200, 1e10, 128.55, 128.57, -3.507, -3.495], "
        "[600, 1e10, 128.803, 128.81, -3.653, -3.644], "
        "[600, 1e10, 128.91, 128.927, -3.352, -3.328]], " )
g.write( "\"AMR_levels\": " + str(AMR_levels))
g.write( "} ")
g.close()

#Format FG Max File
f = open('../../InputData/fgmax_grid.txt', 'w')
f.write('1.0000000000e+01            # tstart_max\n')
f.write('1.0000000000e+10            # tstart_max\n')
f.write('0.0000000000e+00            # dt_check\n')
f.write('3                           # min_level_check\n')
f.write('5.0000000000e-01            # arrival_tol\n')
f.write('0                           # point_style\n')
f.write(str(len(gauges)) + '         # npts\n')

for gauge in gauges:
    f.write(str(gauge['longitude']) + '       ' + str(gauge['latitude']) + '\n')

f.close()

# This Builds The Gauge likelihoods
buildGaugeLikelihoods()